# Open-Source only RAG

- Sentence Transformers as embedding model
- Postgres as vector store
- Llama 2 as LLM
- LLaVa for image summarization

#### Sentence Transformers

In [1]:
#%pip install llama-index-readers-file pymupdf
#%pip install llama-index-vector-stores-postgres
#%pip install llama-index-embeddings-huggingface
#%pip install llama-index-llms-llama-cpp

In [2]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

/Users/busraoguzoglu/Desktop/Okul/586Project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Llama CPP

In [3]:
#!pip install llama-cpp-python

In [4]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

/Users/busraoguzoglu/Desktop/Okul/586Project/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_url" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/busraoguzoglu/Desktop/Okul/586Project/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_path" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/busraoguzoglu/Desktop/Okul/586Project/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
llama_model_loader: loaded meta data

#### Postgres

In [5]:
#!pip install psycopg2-binary pgvector asyncpg "sqlalchemy[asyncio]" greenlet

In [6]:
import psycopg2

# Define your database connection parameters
db_name = "rag_db"  # Use the existing database name here
host = "localhost"
password = "password"  # Replace with your actual PostgreSQL password
port = "5432"  # Default PostgreSQL port
user = "myuser"  # Replace with your actual PostgreSQL username

# Connect directly to the 'rag_db' database
conn = psycopg2.connect(
    dbname=db_name,  # Connect directly to 'rag_db'
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

# Optionally, perform any operations on 'rag_db' using a cursor
with conn.cursor() as c:
    # Drop and create operations are not needed if the database already exists
    # Here you can perform other database setup actions if necessary
    print(f"Connected to {db_name} successfully.")

# Close the connection when done
#conn.close()

Connected to rag_db successfully.


Create table (checks if exist, truncates if already exist and generates again)

In [7]:
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError
from llama_index.vector_stores.postgres import PGVectorStore

# Connection parameters
db_params = {
    "user": "myuser",      # Replace with your actual username
    "password": "password",  # Replace with your actual password
    "host": "localhost",   # Adjust if your database is hosted elsewhere
    "port": "5432",        # Default PostgreSQL port
    "database": "rag_db"   # Replace with your actual database name
}

# Create SQLAlchemy engine
engine = create_engine(f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

# Update table name to include 'data_' prefix
table_name = "data_pmc_table"  # Postgre adds 'data' in front of table_name
embed_dim = 384  # Embedding dimension

try:
    with engine.connect() as conn:
        # Check if the table exists
        result = conn.execute(text(f"""
            SELECT EXISTS (
                SELECT FROM information_schema.tables 
                WHERE table_schema = 'public' AND table_name = :table_name
            );
        """), {"table_name": table_name}).scalar()

        if result:
            print(f"Table '{table_name}' exists. Cleaning it...")
            conn.execute(text(f"TRUNCATE TABLE {table_name};"))
        else:
            print(f"Table '{table_name}' does not exist. Creating it...")

        # Create a new table for vector store
        vector_store = PGVectorStore.from_params(
            database=db_params["database"],
            host=db_params["host"],
            password=db_params["password"],
            port=db_params["port"],
            user=db_params["user"],
            table_name="pmc_table",  # Use unprefixed name; PGVectorStore adds 'data_' automatically
            embed_dim=embed_dim,
        )
        print(f"Table '{table_name}' has been created.")
except OperationalError as e:
    print(f"Error: {e}")
    print("Make sure the database exists and connection parameters are correct.")

Table 'data_pmc_table' exists. Cleaning it...
Table 'data_pmc_table' has been created.


## Ingestion Pipeline

### Data Loading and Preprocessing:

Preprocessing of the document does this:

1- Remove all text after the stopwords, like 'References', because they do not have any information for us.

2- Remove section numbering, letters, extra spaces, new lines.

3- Remove 'tables' from the text.

In [8]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.node_parser import SentenceSplitter
import re
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader
import fitz  # PyMuPDF for image extraction

### Text Preprocessing

In [9]:
# Define stop keywords
stop_keywords = ["References", "REFERENCES", "references", "Conflict of interest", "Conflicts of Interest", "Conflicts"]

# Regex to capture figures
figure_pattern = r'\b(Figure|Fig\.)\s*\d+[a-zA-Z]*\b.*?(\n.*?)*?(?=\n\n|\Z)'

def preprocess_text(text, stop_keywords):
    """Preprocess text by stopping at the earliest occurrence of any keyword, ignoring case."""
    earliest_position = len(text)  # Default to the end of the text
    
    for keyword in stop_keywords:
        pattern = rf"\b{re.escape(keyword)}\b"  # Match exact word boundaries
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            earliest_position = min(earliest_position, match.start())
    
    return text[:earliest_position]

def clean_text(block_text):
    """Clean and process individual blocks of text."""
    block_text = " ".join(block_text.split())  # Remove extra spaces and newlines
    block_text = re.sub(r'^\d+\.\s+', '', block_text)  # Remove section numbering
    block_text = re.sub(r'[^A-Za-z0-9\s,.]', '', block_text)  # Keep only basic punctuation and letters

    if len(block_text) < 10 or re.match(r'Page \d+', block_text):
        return None  # Skip short blocks or page numbers

    if re.search(r'\d{2,}', block_text) and re.search(r'(\d+\.\d+|\d+%)', block_text):
        return None  # Skip blocks with many numeric values

    if re.search(r'\b(Table|Effect|Mediation|Summary|IV|Mediator|Cont)\b', block_text, re.IGNORECASE):
        return None

    return block_text

### Extract figures and save them individually

In [10]:
def extract_figures(text):
    """Extract figure captions or descriptions from the text."""
    figures = re.findall(figure_pattern, text, re.IGNORECASE | re.DOTALL)
    return [" ".join(fig).strip() for fig in figures if len(" ".join(fig).strip()) > 5]  # Filter out short/no-content matches

def extract_images_from_pdf(pdf_path, output_dir, pdf_name):
    """Extract images from a PDF and save them to output_dir."""
    document = fitz.open(pdf_path)
    for page_idx in range(len(document)):
        page = document.load_page(page_idx)
        images = page.get_images(full=True)
        
        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = document.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            
            # Save the image
            image_filename = output_dir / f"{pdf_name}_page{page_idx}_img{img_index}.{image_ext}"
            with image_filename.open("wb") as img_file:
                img_file.write(image_bytes)
            
            print(f"Saved image: {image_filename}")

    document.close()

### Using preprocess functions on documents

In [11]:
# Load and preprocess documents
folder_path = Path("./data")
documents = []

output_path = Path("./processed_documents.txt")  # File to save processed documents
figures_output_path = Path("./extracted_figures_summary.txt")  # Summary of extracted figures
figures_dir = Path("./extracted_figures")  # Directory to save each figure individually
figures_dir.mkdir(exist_ok=True)

with output_path.open("w", encoding="utf-8") as output_file, figures_output_path.open("w", encoding="utf-8") as figures_file:
    for pdf_file in folder_path.glob("*.pdf"):
        loader = PyMuPDFReader()
        loaded_docs = loader.load(file_path=str(pdf_file))
        print(f"{pdf_file.name}: {len(loaded_docs)} documents loaded")

        # Extract images directly from the PDF
        extract_images_from_pdf(pdf_file, figures_dir, pdf_file.stem)

        for doc_idx, doc in enumerate(loaded_docs):
            # Preprocess text
            processed_text = preprocess_text(doc.text, stop_keywords)
            
            # Extract figures from processed text
            figures = extract_figures(processed_text)

            for i, figure in enumerate(figures):
                figure_file_path = figures_dir / f"{pdf_file.stem}_doc{doc_idx}_fig{i}.txt"
                figure_file_path.write_text(figure, encoding="utf-8")  # Write figure caption to its own file

                # Write to the summary file
                figures_file.write(f"{pdf_file.name} - Document {doc_idx}:\n")
                figures_file.write(figure + "\n")
                figures_file.write("=" * 80 + "\n")
            
            print(f"Saved {len(figures)} figure captions for Document {doc_idx} from {pdf_file.name}")

            # Remove figures from text after extraction to clean up
            for figure in figures:
                processed_text = processed_text.replace(figure, "")

            # Clean and process the remaining text blocks
            cleaned_blocks = []
            for block in processed_text.split("\n"):
                cleaned_block = clean_text(block)
                if cleaned_block:
                    cleaned_blocks.append(cleaned_block)
            
            final_text = "\n".join(cleaned_blocks)
            doc.text = final_text  # Replace with cleaned and processed text
            doc.metadata = {"source": pdf_file.name}  # Add metadata
            documents.append(doc)
            
            # Write the processed text into the output file
            output_file.write(f"Document {doc_idx} from {pdf_file.name}:\n")
            output_file.write(final_text + "\n")
            output_file.write("=" * 80 + "\n")  # Separator between documents

print(f"Processed documents written to {output_path}")
print(f"Extracted figure captions written to {figures_output_path}")
print(f"Individual figure files and images saved in '{figures_dir}'")

5.pdf: 10 documents loaded
Saved image: extracted_figures/5_page0_img0.jpeg
Saved image: extracted_figures/5_page1_img0.jpeg
Saved image: extracted_figures/5_page2_img0.jpeg
Saved image: extracted_figures/5_page3_img0.jpeg
Saved image: extracted_figures/5_page4_img0.png
Saved image: extracted_figures/5_page4_img1.jpeg
Saved image: extracted_figures/5_page4_img2.jpeg
Saved image: extracted_figures/5_page5_img0.jpeg
Saved image: extracted_figures/5_page6_img0.jpeg
Saved image: extracted_figures/5_page7_img0.jpeg
Saved image: extracted_figures/5_page8_img0.jpeg
Saved image: extracted_figures/5_page9_img0.jpeg
Saved 0 figure captions for Document 0 from 5.pdf
Saved 0 figure captions for Document 1 from 5.pdf
Saved 0 figure captions for Document 2 from 5.pdf
Saved 0 figure captions for Document 3 from 5.pdf
Saved 0 figure captions for Document 4 from 5.pdf
Saved 0 figure captions for Document 5 from 5.pdf
Saved 0 figure captions for Document 6 from 5.pdf
Saved 0 figure captions for Document

In [12]:
print(documents[65])
print(len(documents))

Doc ID: e9519492-0456-4708-9c0e-477a419bcb1a
Text: MENGI ELIK et al. Roininen, K., Tuorila, H., Zandstra, E., de
Graaf, C., Vehkalahti, K., Stubenitsky, K.,  Mela, D. J. 2001.
Differences in health and taste attitudes and reported behaviour among
Finnish, Dutch and British consumers A crossnational validation of the
health and taste atti tude scales HTAS. Appetite, 371, 3345. Steptoe,
A., Wardle...
66


### Process Pictures

- We will delete the pictures that are smaller than 50KB, since they do not carry any info.

In [13]:
from pathlib import Path

# Define the directory containing extracted images
figures_dir = Path("./extracted_figures")
size_threshold = 50 * 1024  # 50 KB in bytes

# Iterate through all files in the directory
for file in figures_dir.iterdir():
    if file.is_file() and file.suffix.lower() in {'.png', '.jpeg', '.jpg'}:  # Check for image files
        file_size = file.stat().st_size
        if file_size < size_threshold:
            print(f"Deleting {file.name} (Size: {file_size / 1024:.2f} KB)")
            file.unlink()  # Delete the file

print("Cleanup complete: Deleted all image files below 50KB.")

Deleting 5_page0_img0.jpeg (Size: 40.48 KB)
Deleting 4_page7_img0.jpeg (Size: 24.49 KB)
Deleting 5_page4_img0.png (Size: 1.33 KB)
Deleting 3_page14_img0.png (Size: 3.06 KB)
Deleting 5_page6_img0.jpeg (Size: 12.19 KB)
Deleting 5_page1_img0.jpeg (Size: 12.19 KB)
Deleting 3_page0_img1.jpeg (Size: 1.99 KB)
Deleting 5_page4_img2.jpeg (Size: 12.19 KB)
Deleting 5_page7_img0.jpeg (Size: 12.19 KB)
Deleting 3_page0_img2.jpeg (Size: 3.38 KB)
Deleting 5_page2_img0.jpeg (Size: 12.19 KB)
Deleting 5_page9_img0.jpeg (Size: 12.19 KB)
Deleting 1_page0_img0.png (Size: 6.30 KB)
Deleting 4_page0_img1.png (Size: 11.99 KB)
Deleting 4_page0_img0.png (Size: 4.41 KB)
Deleting 3_page0_img3.png (Size: 15.12 KB)
Deleting 4_page0_img2.png (Size: 12.10 KB)
Deleting 5_page3_img0.jpeg (Size: 12.19 KB)
Deleting 4_page0_img3.png (Size: 0.76 KB)
Deleting 3_page0_img0.png (Size: 1.33 KB)
Deleting 5_page8_img0.jpeg (Size: 12.19 KB)
Deleting 4_page0_img4.png (Size: 6.30 KB)
Deleting 5_page5_img0.jpeg (Size: 12.19 KB)
Cleanu

### Splitting Documents

In [14]:
from llama_index.core.node_parser import SentenceSplitter

text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [15]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    #print(f"Document {doc_idx} has {len(cur_text_chunks)} chunks")
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))  

In [16]:
print(len(text_chunks))

88


### Construct Nodes from Text Chunks

In [17]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
        metadata={
            **documents[doc_idxs[idx]].metadata,  # This now includes 'source'
            "doc_index": doc_idxs[idx]  # Optional: document index
        }
    )
    nodes.append(node)

In [18]:
nodes[1]

TextNode(id_='b1f79b11-1efb-4ed9-a7ec-030cd231e26b', embedding=None, metadata={'source': '5.pdf', 'doc_index': 1}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='This change in the lifestyle of the population is partly, due to\nthe need for measures to contain the spread of COVID19, such\nas social isolation. This has proven to be effective8 and has led\nmany people to perform their work remotely.9 This mobility\nrestriction has had direct effects on psychological factors, such\nas an increase in cases of anxiety and depression and a reduction\nin the practice of physical activities.1012 In addition, eating\nhabits were also inuenced both by economic factors, due to the\nreduction in the populations income, as well as by the\nconsumption of foods with higher energy density.13,14\nIn the period before the pandemic, the consumption of fresh\nand minimally processed foods represented approximately 70\nof the total caloric intake by the Brazilian po

### Generate Embeddings for each Node (Text Nodes)

Using sentence_transformers

In [19]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="text")
    )
    node.embedding = node_embedding

In [20]:
print(type(nodes[0].embedding))
print(nodes[0].embedding)
print(len(nodes))

<class 'list'>
[0.002242590067908168, 0.025284932926297188, 0.03787685185670853, 0.017316807061433792, 0.035318728536367416, 0.045880142599344254, 0.02748068794608116, -0.004848837852478027, 0.00520316930487752, -0.02245437726378441, -0.012843466363847256, -0.07361365854740143, 0.0024977654684334993, 0.016995027661323547, 0.0012620296329259872, -0.028180761262774467, -0.0007740127039141953, -0.02978023886680603, 0.00105327891651541, 0.019632909446954727, -0.015793396160006523, -0.012700777500867844, -0.015133718959987164, -0.005372608546167612, 0.041983943432569504, 0.017965346574783325, -0.01487655844539404, -0.03357880935072899, -0.07126981765031815, -0.20361019670963287, -0.021399321034550667, -0.03841941058635712, 0.03099704347550869, -0.01860302872955799, -0.057519495487213135, -0.005936900153756142, 0.03111499361693859, 0.016658559441566467, -0.00020628668426070362, -0.011791287921369076, 0.0005544592277146876, 0.042607758194208145, 0.04287157580256462, -0.04061812534928322, -0.0

### Generate Embeddings for Images

- We will use LLaVA model to generate summaries from images.

In [21]:
from transformers import AutoProcessor, LlavaForConditionalGeneration
from PIL import Image
import torch
from pathlib import Path

# Load LLaVA processor and model
model_name = "llava-hf/llava-1.5-7b-hf"
processor = AutoProcessor.from_pretrained(model_name)
model = LlavaForConditionalGeneration.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 
Loading checkpoint shards: 100%|██████████| 3/3 [00:10<00:00,  3.51s/it]


In [22]:
# Check device and use it
device = "mps" if torch.has_mps else "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Directory of extracted figures
figures_dir = Path("./extracted_figures")
image_summaries = {}

# Generate summaries for each image (both .png and .jpeg)
for image_file in figures_dir.glob("*"):
    if image_file.suffix.lower() not in [".png", ".jpeg", ".jpg"]:
        continue  # Skip files that are not images

    image = Image.open(image_file).convert("RGB")
    
    # Define the conversation template for generating a detailed description
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "image"},  # Image placeholder
                {"type": "text", "text": "What is shown in this image?"}
            ],
        }
    ]
    
    # Apply chat template and prepare inputs
    text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
    inputs = processor(text=[text_prompt], images=[image], return_tensors="pt").to(device)  # Ensure inputs are on the same device
    
    # Generate response
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=256)
    
    summary = processor.decode(outputs[0], skip_special_tokens=True)
    
    # Extract only the assistant's response
    assistant_reply = summary.split("ASSISTANT:")[1].strip() if "ASSISTANT:" in summary else summary
    image_summaries[image_file.name] = assistant_reply
    print(f"Generated summary for {image_file.name}:\n{assistant_reply}\n")

# Save summaries to file
summary_output_path = Path("./image_summaries.txt")
with summary_output_path.open("w", encoding="utf-8") as f:
    for image_name, summary in image_summaries.items():
        f.write(f"{image_name}:\n{summary}\n{'='*80}\n")

print(f"Image summaries saved to {summary_output_path}")

/var/folders/ss/9g03411n5sgb8vs4fvgmqlfw0000gp/T/ipykernel_91791/1231273736.py:2: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = "mps" if torch.has_mps else "cuda" if torch.cuda.is_available() else "cpu"
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Generated summary for 1_page4_img1.jpeg:
The image shows a bar graph or a chart with a red line and a green line, representing two different things. The red line is likely a percentage, while the green line is a different measurement. The chart is divided into sections, with each section showing the corresponding values of the two different measurements.

Generated summary for 1_page4_img0.jpeg:
The image shows a chart or graph displaying various health-related statistics, such as the number of calories burned, the number of calories consumed, and the number of calories needed for a healthy diet. The chart is color-coded, with green representing a positive outcome, red representing a negative outcome, and yellow representing a neutral outcome.

Generated summary for 1_page5_img0.jpeg:
The image shows a flowchart or diagram that illustrates the process of healthy eating.

Generated summary for 1_page5_img1.png:
The image shows a diagram or flowchart that illustrates the process of liste

### Embeddings of Image Summaries

In [23]:
from pathlib import Path

# Path to the image summaries file
summaries_path = Path("./image_summaries.txt")

# Dictionary to store image summaries
image_summaries = {}

# Read the summaries file and populate the dictionary
with summaries_path.open("r", encoding="utf-8") as f:
    lines = f.readlines()
    current_image = None
    current_summary = []
    
    for line in lines:
        line = line.strip()
        if line.endswith(":") and not line.startswith("="):  # Detect image name
            if current_image and current_summary:
                image_summaries[current_image] = " ".join(current_summary).strip()
                current_summary = []  # Reset for the next summary
            current_image = line[:-1]  # Remove trailing colon
        elif not line.startswith("="):  # Skip separator lines
            current_summary.append(line)
    
    # Add the last image and its summary
    if current_image and current_summary:
        image_summaries[current_image] = " ".join(current_summary).strip()

# Print the loaded summaries for verification
for image, summary in image_summaries.items():
    print(f"{image}: {summary}")

1_page4_img1.jpeg: The image shows a bar graph or a chart with a red line and a green line, representing two different things. The red line is likely a percentage, while the green line is a different measurement. The chart is divided into sections, with each section showing the corresponding values of the two different measurements.
1_page4_img0.jpeg: The image shows a chart or graph displaying various health-related statistics, such as the number of calories burned, the number of calories consumed, and the number of calories needed for a healthy diet. The chart is color-coded, with green representing a positive outcome, red representing a negative outcome, and yellow representing a neutral outcome.
1_page5_img0.jpeg: The image shows a flowchart or diagram that illustrates the process of healthy eating.
1_page5_img1.png: The image shows a diagram or flowchart that illustrates the process of listening to the body.
1_page8_img0.jpeg: The image shows a chart or graph displaying various st

In [24]:
from llama_index.core.schema import TextNode
from pathlib import Path
from sentence_transformers import SentenceTransformer

# Initialize nodes for image summaries
image_summary_nodes = []

for image_name, summary in image_summaries.items():
    # Generate embedding for the image summary
    node_embedding = embed_model.get_text_embedding(summary)
    
    # Create a TextNode instance
    node = TextNode(
        text=summary,
        embedding=node_embedding,
        metadata={
            "source": image_name,  # The image file name
            "type": "image_summary",  # To distinguish from text nodes
        }
    )
    
    image_summary_nodes.append(node)

# Display one of the nodes for verification
print(image_summary_nodes[0])

Node ID: f7877061-b42b-4e94-bafe-abe2d3041ca6
Text: The image shows a bar graph or a chart with a red line and a
green line, representing two different things. The red line is likely
a percentage, while the green line is a different measurement. The
chart is divided into sections, with each section showing the
corresponding values of the two different measurements.


### Load Nodes (Text Embeddings) and Image Summary Embeddings into a Vector Store

We now insert these nodes into our `PostgresVectorStore`.

In [25]:
vector_store.add(nodes)

['2051e2ef-f22f-4d8e-a03f-711bfdc907c5',
 'b1f79b11-1efb-4ed9-a7ec-030cd231e26b',
 'b1126a10-ecf9-4b90-b692-a35c136dbc3c',
 '892bce11-cadd-4ab0-b416-cc80a90b01fa',
 '2f1a623b-8c0e-4db6-a231-6aa90147c6c1',
 'a222c75a-be15-474a-b65c-4192f90ea1b8',
 '057e36c6-2ea9-4c22-93f3-08451627c514',
 '251dcea8-aaa3-4fc7-b219-9869aa0e89b3',
 '01f93d1e-9eb6-4ef1-9ea5-1f98f4c9e761',
 '4ce3538a-e0d9-4555-b24f-a32f7986757a',
 '989a51fd-c76b-4a1d-aff4-0b039a64ae8a',
 '7ed86d1f-e1fd-457b-9b4f-12e07c91a23c',
 'ff388789-7b38-496f-b86a-4757474ea996',
 '16e1b4d8-ca1b-4e9e-ada7-2b636e77554a',
 '29b173a0-fad4-46b5-84b1-95d92e841f76',
 '6c9d3302-b554-492b-a3b3-f086975f444d',
 '314d9517-4094-4955-aaac-64573b5a46a3',
 'c7799e42-0098-4e1c-8fff-add87b286cc0',
 '4ccd8e7a-9906-40d9-887f-b05e0789b94a',
 'e382672a-0e2c-40e4-8997-3e3575378724',
 '8c8755ed-cb66-4729-a48d-ae3c66d73a3c',
 '8c930c52-90ec-4370-bff5-9cc79c42d099',
 'a1ccf93a-405d-47f0-b3e0-73b79acc75b4',
 '9419062e-327e-4491-8648-e150ea99fc49',
 '71f28e7f-9edd-

In [26]:
vector_store.add(image_summary_nodes)

['f7877061-b42b-4e94-bafe-abe2d3041ca6',
 '3601bb1a-7768-48df-b708-84bcc34b63c9',
 'b9231d65-637b-4df8-90c8-d59e10ffded5',
 '7e63b448-8ec3-4a1c-aaf2-35c7234fa5c0',
 '6833515f-20c1-4214-9d67-a584a50389e9',
 'ce09d5de-fc47-4547-82c4-3be3e1332e46',
 '6f52d4ff-6f11-46c7-9de3-3ea3567acb68',
 'ad8ebe79-33d9-4f39-9595-976675172ef8']

In [27]:
# Create SQLAlchemy engine
engine = create_engine(f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

# Query to count rows in the vector store table
table_name = "data_pmc_table"  # Adjust the table name as necessary

with engine.connect() as conn:
    result = conn.execute(text(f"SELECT COUNT(*) FROM {table_name};"))
    node_count = result.scalar()
    print(f"Number of nodes in vector store: {node_count}")

Number of nodes in vector store: 1084


## Retrieval Pipeline

In [28]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List
from llama_index.core.schema import NodeWithScore
from typing import Optional
from llama_index.core.vector_stores import VectorStoreQuery

In [29]:
class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [30]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

retriever

## Response

In [31]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [32]:
query_str = "Which individuals play a central role in promoting healthy eating?"

response = query_engine.query(query_str)


llama_print_timings:        load time =   15851.78 ms
llama_print_timings:      sample time =       0.37 ms /    14 runs   (    0.03 ms per token, 37837.84 tokens per second)
llama_print_timings: prompt eval time =   15851.03 ms /   476 tokens (   33.30 ms per token,    30.03 tokens per second)
llama_print_timings:        eval time =    1860.26 ms /    13 runs   (  143.10 ms per token,     6.99 tokens per second)
llama_print_timings:       total time =   17717.95 ms /   489 tokens


In [33]:
print(str(response))



Please provide the answer based on the given context information.


In [34]:
print(response.source_nodes[0].get_content())

An exploratory and descriptive crosssectional study
was conducted using a qualitative and quantitative methodology and convenience sampling. Two
focus groups and a questionnaire were utilized 300 participants from all academic years completed
the survey. Differences in definitions of healthy eating and perceived barriers were found between
genders and students at different stages of training p  0.05. In their understanding of healthy eating,
the students placed importance on balance, variety, moderation, and individual factors. Although
students considered it easy to follow a healthy diet, familys eating habits, time availability, and
emotional states were found to be the main barriers to the implementation of healthy practices. The
obtained data supports the need to critically address perceptions of healthy eating throughout the
training of nutrition and food science professionals. The insights obtained on the perceived barriers
highlight the importance of considering both individual 

In [35]:
query_str = "What does nutrition literacy significantly predicted in the social-ecological framework?"

response = query_engine.query(query_str)

Llama.generate: 13 prefix-match hit, remaining 1253 prompt tokens to eval

llama_print_timings:        load time =   15851.78 ms
llama_print_timings:      sample time =       3.07 ms /   119 runs   (    0.03 ms per token, 38711.78 tokens per second)
llama_print_timings: prompt eval time =   28273.47 ms /  1253 tokens (   22.56 ms per token,    44.32 tokens per second)
llama_print_timings:        eval time =   13580.93 ms /   118 runs   (  115.09 ms per token,     8.69 tokens per second)
llama_print_timings:       total time =   41916.12 ms /  1371 tokens


In [36]:
print(str(response))

 Based on the provided context information, nutrition literacy significantly predicted adherence to healthy-unhealthy diet patterns in adults with a nutrition-related chronic condition, as well as health-promoting behaviors among college students in Taiwan. Additionally, nutrition literacy was found to be associated with eating habits and BMI in adolescents in Lebanon. Overall, nutrition literacy appears to play a significant role in predicting health-promoting behaviors and dietary patterns in various populations.


In [37]:
response.source_nodes[0]

NodeWithScore(node=TextNode(id_='bccff988-f457-4cad-946b-037732a0d87c', embedding=None, metadata={'source': '4.pdf', 'doc_index': 20}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Lai, I.J. Chang, L.C. Nutrition literacy is associated with healthyeating behaviour among college students in Taiwan.\nHealth Educ. J. 2019, 78, 756769. CrossRef\nYuen, E. Thomson, M. Gardiner, H. Measuring Nutrition and Food Literacy in Adults A Systematic Review and Appraisal of\nExisting Measurement Tools. HLRP Health Lit. Res. Pract. 2018, 2, e134e160. CrossRef PubMed\nSilk, K.J. Sherry, J. Winn, B. Keesecker, N. Horodynski, M.A. Sayir, A. Increasing Nutrition Literacy Testing the Effectiveness\nof Print, Web site, and Game Modalities. J. Nutr. Educ. Behav. 2008, 40, 310. CrossRef\nAihara, Y. Minai, J. Barriers and catalysts of nutrition literacy among elderly Japanese people. Health Promot. Int. 2011, 26,\n421431. CrossRef\nZoellner, J. Connell, C. Bounds, W. Cr

In [38]:
print(response.source_nodes[0].get_content())

Lai, I.J. Chang, L.C. Nutrition literacy is associated with healthyeating behaviour among college students in Taiwan.
Health Educ. J. 2019, 78, 756769. CrossRef
Yuen, E. Thomson, M. Gardiner, H. Measuring Nutrition and Food Literacy in Adults A Systematic Review and Appraisal of
Existing Measurement Tools. HLRP Health Lit. Res. Pract. 2018, 2, e134e160. CrossRef PubMed
Silk, K.J. Sherry, J. Winn, B. Keesecker, N. Horodynski, M.A. Sayir, A. Increasing Nutrition Literacy Testing the Effectiveness
of Print, Web site, and Game Modalities. J. Nutr. Educ. Behav. 2008, 40, 310. CrossRef
Aihara, Y. Minai, J. Barriers and catalysts of nutrition literacy among elderly Japanese people. Health Promot. Int. 2011, 26,
421431. CrossRef
Zoellner, J. Connell, C. Bounds, W. Crook, L. Yadrick, K. Nutrition Literacy Status and Preferred Nutrition Communication
Channels among Adults in the Lower Mississippi Delta. Prev. Chronic Dis. 2009, 6, A128.
Noroozi, A. Khademolhosseini, F. Lari, H. Tahmasebi, R. The